In [20]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import sympy

In [126]:
N = 200
phi = 0.25
GEN = 50
TOP = 10
CLONE = 0.1

In [39]:
def h(x):
  return -x * (x-0.25) * (x-1)

In [104]:
class Individual:
  def __init__(self, parent=False, clone=False, par=None, mutations=2):
    self.weights = np.random.uniform(-1,1, (8,))
    self.syms = sympy.symbols("A:H")
    self.phi = sympy.Symbol("phi")
    self.eq = self.generate_eq()
    self.mutations = mutations
    self.y = 0.25
    if parent:
      self.from_parent(clone, par)
  
  def from_parent(self,clone:bool, par: "Individual"):
    self.weights = par.weights.copy()
    if not clone:
      rand = np.random.randint(0,8, (self.mutations))
      for index in rand:
        self.weights[index] = np.random.uniform(-1,1)
    
  
  def ret_eq(self, params):
    roots = sympy.real_roots(self.eq.subs(params))
    all = []
    for i in roots:
      all.append(i.evalf())
    ind = np.abs(np.array(all) - 0.25).argmin()
    sign = self.eq.subs(params).diff().subs({self.phi: all[ind]})
    if sign > 0:
      return all[ind] * 10
    return all[ind]
  
  def stabilities(self):
    params = dict(zip(self.syms, self.weights))
    return float(self.ret_eq(params))

  def return_matrices(self):
    gamma = np.zeros((2,2)).tolist()
    alpha = np.zeros_like(gamma).tolist()
    for ind,w in enumerate(self.weights[:4]):
      gamma[ind>1][ind%3 != 0] = w
    for ind,w in enumerate(self.weights[4:]):
      alpha[ind >1][ind%3 != 0] = w
    return alpha, gamma

  def from_matrices(self, alpha, gamma):
    out = []
    for ind in range(4):
      out.append(gamma[ind > 1][ind%3!=0])
    for ind in range(4):
      out.append(alpha[ind> 1][ind%3!=0])
    self.weights = np.array(out)

  def generate_eq(self):
    eq = 0
    # h = lambda n: -n*(n-0.25) * (n-1)

    for i,val in enumerate(self.syms):
      add = ((i % 2) == 1) * 0.5
      neg = (i > 3) * 2 - 1
      eq+=h(neg* self.phi + add) * val * -1 * neg
    return eq
  
  def print_eq(self):
    params = dict(zip(self.syms, self.weights))
    return self.eq.subs(params).subs({self.phi: sympy.Symbol("x")})
  
  def reset(self):
    self.weights = np.random.uniform(-1,1,(8,)).astype(np.float32)
    return self.weights
 

In [103]:
Individual().print_eq().diff().subs({"x":0.25})

1.75551522232452

In [128]:
for i in range(20):
    individuals = [Individual() for i in range(N)]

    for _ in tqdm(range(GEN)):
        steps = [ind.stabilities() for ind in individuals]
        
        normed = np.absolute(np.array(steps,dtype=float) - phi)
        sorted = np.argsort(normed)[:TOP]
        
        topN = np.array(individuals)[sorted.astype(int)]

        individuals.clear()

        totalper = N/TOP
        numper = (200 * CLONE) // TOP

        for ind in topN:
            for _ in range(int(numper)):
                individuals.append(Individual(parent=True, clone=True, par=ind))
            for _ in range(int(totalper-numper)):
                individuals.append(Individual(parent=True, clone=False, par=ind))
    out = np.concatenate(topN[0].return_matrices())
    np.save("gen_trials.npy",out)

 50%|█████     | 25/50 [06:46<06:46, 16.26s/it]


KeyboardInterrupt: 

In [127]:
np.concatenate(topN[0].return_matrices())

array([[ 0.99647353, -0.54286653],
       [-0.45824586,  0.98869601],
       [-0.44353052,  0.9720073 ],
       [ 0.98677499, -0.15714826]])